# Dual Thrust 策略

## 摘要

## 理论模型

### 日内策略



In [ ]:
# coding=utf-8
"""Dual Thrust 策略"""
import datetime as dt

import numpy as np
import pandas as pd

import QuantStudio.api as QS

# Dual Thrust 日内策略
# 计算:
# (1) 前 N 日最高价的最大值 HH, 前 N 日收盘价的最小值 LC, 前 N 日收盘价的最大值 HC, 前 N 日最低价的最小值 LL;
# (2) Range = max(HH-LC, HC-LL)
# (3) BuyLine = Open + K1 * Range
# (4) SellLine = Open - K2 * Range
# 交易逻辑:
# (1) 当价格向上突破 BuyLine 时, 如果当时持有空仓, 则先平仓, 再开多仓; 如果没有仓位, 则直接开多仓;
# (2) 当价格向下突破 SellLine 时, 如果当时持有多仓, 则先平仓, 再开空仓; 如果没有仓位, 则直接开空仓;
# (3) 不持仓过夜
class DualThrustStrategy(QS.BackTest.Strategy.TimingStrategy):
    def init(self):
        self.UserData["Dates"] = self.FactorTables[1].getDateTime()# 提取日期序列, 用于计算上下轨
        self.UserData["CurDate"] = None
        self.UserData["DateIndex"] = None
        self.UserData["BuyLine"] = None
        self.UserData["SellLine"] = None
    def genSignal(self, idt, trading_record):
        TargetIDs = self.TargetIDs
        if idt.date()!=self.UserData["CurDate"]:# 进入新的日期, 重新计算上下轨
            self.UserData["CurDate"] = idt.date()
            iDT = dt.datetime.combine(self.UserData["CurDate"], dt.time(0))
            if iDT in self.UserData["Dates"]: self.UserData["DateIndex"] = self.UserData["Dates"].index(iDT)
            if (self.UserData["DateIndex"] is None) or (self.UserData["DateIndex"]<self.ModelArgs["N_Buy"]) or (self.UserData["DateIndex"]<self.ModelArgs["N_Sell"]): return None
            DTs = self.UserData["Dates"][self.UserData["DateIndex"]-max(self.ModelArgs["N_Buy"], self.ModelArgs["N_Sell"]):self.UserData["DateIndex"]]
            Price = self.FactorTables[1].readData(factor_names=["最高价", "最低价", "收盘价"], ids=self.ModelArgs["TargetIDs"], dts=DTs)
            Open = self.FactorTables[1].readData(factor_names=["开盘价"], ids=self.ModelArgs["TargetIDs"], dts=[dt.datetime.combine(idt.date(), dt.time(0))]).iloc[0,0,:]
            HH, HC, LC, LL = Price["最高价"].iloc[-self.ModelArgs["N_Buy"]:].max(), Price["收盘价"].iloc[-self.ModelArgs["N_Buy"]:].max(), Price["收盘价"].iloc[-self.ModelArgs["N_Buy"]:].min(), Price["最低价"].iloc[-self.ModelArgs["N_Buy"]:].min()
            BuyRange = np.maximum(HH-LC, HC-LL)
            self.UserData["BuyLine"] = Open + self.ModelArgs["K_Buy"] * BuyRange
            HH, HC, LC, LL = Price["最高价"].iloc[-self.ModelArgs["N_Sell"]:].max(), Price["收盘价"].iloc[-self.ModelArgs["N_Sell"]:].max(), Price["收盘价"].iloc[-self.ModelArgs["N_Sell"]:].min(), Price["最低价"].iloc[-self.ModelArgs["N_Sell"]:].min()
            SellRange = np.maximum(HH-LC, HC-LL)
            self.UserData["SellLine"] = Open - self.ModelArgs["K_Sell"] * SellRange
            self.UserData["BuyLine"].index, self.UserData["SellLine"].index = TargetIDs, TargetIDs
        elif idt.time()>=dt.time(14, 57):# 最后三分钟进行平仓
            return pd.Series(0.0, index=TargetIDs)
        Price = self.MainFactorTable.readData(factor_names=["price"], ids=TargetIDs, dts=[idt]).iloc[0,0,:]
        Signal = pd.Series(np.nan, index=TargetIDs)
        Signal[Price>self.UserData["BuyLine"]] = 1
        Signal[Price<self.UserData["SellLine"]] = -1
        return Signal

if __name__=='__main__':
    # 创建因子库
    ADB = QS.FactorDB.ArcticDB()
    ADB.connect()
    
    HDB = QS.FactorDB.HDF5DB()
    HDB.connect()
    
    # 创建因子表
    FT = ADB.getTable("MinuteBar")
    DTs = FT.getDateTime(ifactor_name="price", start_dt=dt.datetime(2018, 6, 1), end_dt=dt.datetime(2018, 10, 1))
    FT["遍历模式"]["缓冲模式"] = "ID"
    FT["遍历模式"]["遍历时点"] = DTs
    IDs = ["IF00"]
    
    # 创建回测模型
    Model = QS.BackTest.BackTestModel()
    
    iAccount = QS.BackTest.Strategy.DefaultAccount(market_ft=FT)
    iAccount["初始资金"] = 1e8
    iAccount["负债上限"] = 0.0
    iAccount["交易延迟"] = False
    iAccount["目标ID"] = IDs
    iAccount["买入限制"]["交易费率"] = 0.0005
    iAccount["卖出限制"]["交易费率"] = 0.0005
    iAccount["卖出限制"]["允许卖空"] = True
    iAccount["行情信息"]["成交价"] = "price"
    iAccount["行情信息"]["最新价"] = "price"
    
    iModule = DualThrustStrategy(name="DualThrustStrategy", factor_table=FT)
    iModule["目标账户"] = iAccount
    iModule["资金分配"] = pd.Series(iAccount["初始资金"]/len(IDs), index=IDs)
    DayFT = HDB.getTable("IndexFutureContinuousContractFactor")
    DayFT["遍历模式"]["向后缓冲时点数"] = 10
    iModule.FactorTables.append(DayFT)# 加入日行情因子表
    iModule.ModelArgs["TargetIDs"] = ["IF.CFE"]# 日行情表中对应的目标 ID
    iModule.ModelArgs["N_Buy"] = 5
    iModule.ModelArgs["N_Sell"] = 5
    iModule.ModelArgs["K_Buy"] = 0.7
    iModule.ModelArgs["K_Sell"] = 0.7
    Model.Modules.append(iModule)
    
    # 运行模型
    Model.run(dts=DTs)
    
    # 查看结果
    QS.Tools.QtGUI.showOutput(Model.output())